In [ ]:
import pandas
pandas.__version__

In [ ]:
pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', None)

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
df = pandas.read_csv('../../v3_CSV/databases/connections_database.csv', 
                     header=None,
                     sep="\s*,\s*", engine='python',
                     names=['derivation name', 'linear index', 
                            'input type', 'in local ID', 'in global ID',
                            'output type', 'out local ID', 'out global ID'])

In [ ]:
df['derivation name'] = df['derivation name'].str.replace('"', '')
df['input type'] = df['input type'].str.replace('"', '')
df['output type'] = df['output type'].str.replace('"', '')

In [ ]:
df.shape

In [ ]:
df.head(23)

In [ ]:
list_of_derivs = list(df['derivation name'].unique())

# single derivation; "expr local to global"

In [ ]:
this_deriv_df = df[df['derivation name']==list_of_derivs[0]]

In [ ]:
def get_expr_tuples_for_df(this_deriv_df):
    list_of_expr_tuples = []
    for indx, ths_row in this_deriv_df.iterrows():
        row_as_dict = ths_row.to_dict()
        if row_as_dict['input type'] == 'expression':
            expr_tuple = (str(row_as_dict['in local ID']), row_as_dict['in global ID'])
            list_of_expr_tuples.append(expr_tuple)
        if row_as_dict['output type'] == 'expression':
            expr_tuple = (str(row_as_dict['out local ID']), row_as_dict['out global ID'])
            list_of_expr_tuples.append(expr_tuple)
    return list(set(list_of_expr_tuples))

In [ ]:
list_of_expr_tuples = get_expr_tuples_for_df(this_deriv_df)
list_of_expr_tuples

# single derivation; "derivations"

In [ ]:
list_of_derivs[0]

In [ ]:
this_deriv_df = df[df['derivation name']==list_of_derivs[0]]

In [ ]:
def get_step_IDs(this_deriv_df):
    list_of_step_IDs = []
    for indx, ths_row in this_deriv_df.iterrows():
        row_as_dict = ths_row.to_dict()
        if row_as_dict['input type']=='infrule':
            list_of_step_IDs.append(row_as_dict['in local ID'])
        elif row_as_dict['output type']=='infrule':
            list_of_step_IDs.append(row_as_dict['out local ID'])

    return list(set(list_of_step_IDs))

In [ ]:
list_of_step_IDs = get_step_IDs(this_deriv_df)

In [ ]:
list_of_step_IDs

In [ ]:
def extract_printable_steps(list_of_step_IDs, this_deriv_df):
    dict_of_steps = {}
    for step_ID in list_of_step_IDs:
        step_str=""
        #print('step_ID =', step_ID)
        list_of_dicts_for_this_step = []
        for indx, ths_row in this_deriv_df.iterrows():
            #print(ths_row.to_dict())
            row_as_dict = ths_row.to_dict()
            if row_as_dict['input type'] == 'infrule' and row_as_dict['in local ID']==step_ID:
                list_of_dicts_for_this_step.append(row_as_dict)
            if row_as_dict['output type'] == 'infrule' and row_as_dict['out local ID']==step_ID:
                list_of_dicts_for_this_step.append(row_as_dict)

    #    print(list_of_dicts_for_this_step)

        for sub_dict in list_of_dicts_for_this_step:
            if sub_dict['input type']=='infrule':
                inf_rule = sub_dict['in global ID']
            if sub_dict['output type']=='infrule':
                inf_rule = sub_dict['out global ID']
        step_str+=("     '"+str(step_ID)+"': {'inf rule': '"+inf_rule+"',\n")
        
        list_of_input_expr_local_ID = []
        for sub_dict in list_of_dicts_for_this_step:
            if sub_dict['input type']=='expression':
                list_of_input_expr_local_ID.append(sub_dict['in local ID'])
        list_of_input_expr_local_ID = list(set(list_of_input_expr_local_ID))
        # https://stackoverflow.com/questions/3590165/join-a-list-of-items-with-different-types-as-string-in-python
        list_of_input_expr_local_ID = ', '.join("'"+str(x)+"'" for x in list_of_input_expr_local_ID)
        step_str+=("                 'inputs':  ["+str(list_of_input_expr_local_ID)+"],\n" )
        
        list_of_input_expr_local_ID = []
        for sub_dict in list_of_dicts_for_this_step:
            if sub_dict['input type']=='feed':
                list_of_input_expr_local_ID.append(sub_dict['in local ID'])
        list_of_input_expr_local_ID = list(set(list_of_input_expr_local_ID))
        list_of_input_expr_local_ID = ', '.join("'"+str(x)+"'" for x in list_of_input_expr_local_ID)
        step_str+=("                 'feeds':   ["+str(list_of_input_expr_local_ID)+"],\n" )

        list_of_input_expr_local_ID = []
        for sub_dict in list_of_dicts_for_this_step:
            if sub_dict['output type']=='expression':
                list_of_input_expr_local_ID.append(sub_dict['out local ID'])
        list_of_input_expr_local_ID = list(set(list_of_input_expr_local_ID))
        list_of_input_expr_local_ID = ', '.join("'"+str(x)+"'" for x in list_of_input_expr_local_ID)
        step_str+=("                 'outputs': ["+list_of_input_expr_local_ID+"],\n" )

        step_str+=("                 'linear index': "+str(list_of_dicts_for_this_step[0]['linear index'])+"},")

        dict_of_steps[list_of_dicts_for_this_step[0]['linear index']] = step_str

    return dict_of_steps

In [ ]:
dict_of_steps = extract_printable_steps(list_of_step_IDs, this_deriv_df)

In [ ]:
list_of_linindx = list(dict_of_steps.keys())
list_of_linindx.sort()
for linindx in list_of_linindx:
    print(dict_of_steps[linindx])

# all the derivations

In [ ]:
for this_d in list_of_derivs:
    this_deriv_df = df[df['derivation name']==this_d]
    list_of_expr_tuples = get_expr_tuples_for_df(this_deriv_df)
    print('# '+this_d)
    for this_tup in list_of_expr_tuples:
        print("        '"+this_tup[0]+"': '"+this_tup[1]+"',")

In [ ]:
for this_d in list_of_derivs:
    this_deriv_df = df[df['derivation name']==this_d]
    list_of_step_IDs = get_step_IDs(this_deriv_df)
    dict_of_steps = extract_printable_steps(list_of_step_IDs, this_deriv_df)
    list_of_linindx = list(dict_of_steps.keys())
    list_of_linindx.sort()
    print("  '"+this_d+"': {")
    for linindx in list_of_linindx:
        print(dict_of_steps[linindx])
    print("      },")